In [1]:
!pip install hdfs
# ref : https://hdfscli.readthedocs.io/en/latest/quickstart.html#python-bindings

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 KB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for hdfs: filename=hdfs-2.7.3-py3-none-any.whl size=34347 sha256=0bf18877a9b2265c9ccf4d570f5efba8243f7b40aac330d43a6cb5199096778c
  Stored in directory: /root/.cache/pip/wheels/e5/8d/b6/99c1c0a3ac5788c866b0ecd3f48b0134a5910e6ed26011800b
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=8fe7347d523ce5daee37764c7e05eadb5399c692753827ffbc1d014721d00805
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built hdfs docopt


In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
from hdfs import InsecureClient # hdfs 연결 담당 모듈 - 클라이언트 프로그램 구성 시 
import requests # api 요청시 사용
import json # api다운받은 data와 vaccine data json 처리시 사용
import datetime as dt # 날짜 시간관련 기능을 제공하는 클래스

## hdfs 사용 클라이언트 객체 생성
- hdfs 연결 담당
- read/write/append/update 등과 관련된 모듈 포함하고 있음
- InsecureClient(연결주소, user=사용자)
- 파일읽기
    - InsecureClient.read(파일명(파일경로 포함))
        - 해당 파일을 읽을 수 있는 reader 객체를 반환
    - InsecureClient.write(저장될파일명(파일경로 포함))
        - 해당 파일에 쓸 수 있는 writer 객체를 반환
    - InsecureClient.write(저장될파일명(파일경로 포함),'추가될 내용',append=True)
        - 기존파일에 추가될내용을 append(파일 끝에 쓰기)        
    - InsecureClient.upload(hdfs에 복사될 폴더명(경로포함), 원본 폴더명(경로포함))
        - 지정된 원본 폴더 파일을 저장할 hsdf 폴더에 복사
        - 폴더를 생성하고 시작 : 기존에 동일 이름의 폴더가 있으면 에러발생함 

In [4]:
client = InsecureClient('http://localhost:9870', user='root')

## hdfs로 부터 읽기

- 파일 읽어올때
    - 파일과 연결(open)
    - 파일읽기
    - 파일 연결 끊기(close) : 개발자가 close해야 자원 정리가 됨
        - with문 사용하면 자동 open ~ close를 진행함
        - 

In [5]:
# 파일 읽기위한 reader 객체 생성 후 진행
with client.read('/rdd/score.txt') as reader :
    score = reader.read()
    

In [6]:
# byte code 읽어옴
score

b'\xea\xb9\x80\xec\xb2\xa0\xec\x88\x98 \xec\x8a\xa4\xed\x8c\x8c\xed\x81\xac 50\r\n\xed\x99\x8d\xea\xb8\xb8\xeb\x8f\x99 \xec\x8a\xa4\xed\x8c\x8c\xed\x81\xac 80\r\n\xec\x9e\x84\xea\xba\xbd\xec\xa0\x95 \xec\x8a\xa4\xed\x8c\x8c\xed\x81\xac 60\r\n\xec\x9e\x84\xec\x9a\x94\xed\x99\x98 \xed\x85\x90\xec\x84\x9c\xed\x94\x8c\xeb\xa1\x9c\xec\x9a\xb0 100\r\n\xed\x99\x8d\xec\xa7\x84\xed\x98\xb8 \xed\x85\x90\xec\x84\x9c\xed\x94\x8c\xeb\xa1\x9c\xec\x9a\xb0 22\r\n\xed\x99\x8d\xec\xa7\x84\xed\x98\xb8 \xed\x85\x90\xec\x84\x9c\xed\x94\x8c\xeb\xa1\x9c\xec\x9a\xb0 22\r\n\xec\x9d\xb4\xec\x9c\xa4\xec\x97\xb4 \xed\x85\x90\xec\x84\x9c\xed\x94\x8c\xeb\xa1\x9c\xec\x9a\xb0 90\r\n'

In [7]:
# https://docs.python.org/ko/3/library/stdtypes.html#bytes-and-bytearray-operations
# byte code decode해서 저장 후 확인
# bytes.decode(data)
score_data = bytes.decode(score)
print(score_data)

김철수 스파크 50
홍길동 스파크 80
임꺽정 스파크 60
임요환 텐서플로우 100
홍진호 텐서플로우 22
홍진호 텐서플로우 22
이윤열 텐서플로우 90



## hdfs에 쓰기

In [8]:
# 파일쓰기
# hdd(note 디렉터리)에 있는 data(file)를 hdfs로 복사
# 1. 로컬데이터 파일 열기 : open() python method 사용, width 등록
# 2. hdfs client 통해서 쓰기 객체 with 등록
# 3. open 객체 이용해서 한줄씩 읽어서 -> hdfs에 쓰기
# 현재 위치 : /root/note
# 복사할 sido_area.csv 윈도우 10 버전에서 생성, encoding type 'CP949'
with open('data/corona_data/sido_area.csv', encoding='CP949') as reader, client.write('/corona_data/sido_area_tmp.csv') as writer :
    for line in reader : # 읽어와서
        writer.write(line.encode('CP949')) # 원래 encoding 방식 유지 시켜야 함, 바로쓰기

In [9]:
# 폴더 전송 : 폴더내에있는 파일을 hdfs 특정 디렉터리 모두 복사
# 해당 모듈은 디렉터리와 파일부터 생성하고 쓰기 진행
# 프로그램으로 파일 한줄씩 읽어와서 바로 hdfs 파일에 쓰기
# client.upload(dest, source)
client.upload('/corona_data/tmp', 'data/corona_data')

'/corona_data/tmp'

## hdfs 파일 수정하기
- data 추가이므로 data 형식과 encoding 방식은 기존 파일과 동일하게 구성

In [10]:
client.write('/rdd/score.txt','최연성 장고 100'.encode('UTF-8'), append=True)

In [11]:
# 수정된 파일 읽어와서 확인
# reader = client.read('/rdd/score.txt')와 같은 동작이므로 reader는 객체 변수
with client.read('/rdd/score.txt') as reader :
    score = reader.read()
score_data = bytes.decode(score)
print(score_data)

김철수 스파크 50
홍길동 스파크 80
임꺽정 스파크 60
임요환 텐서플로우 100
홍진호 텐서플로우 22
홍진호 텐서플로우 22
이윤열 텐서플로우 90
최연성 장고 100


## hdfs 파일 권한 수정
- 리눅스 명령어(chmod)
- InsecureClient클래스의 set_permission('권한변경할파일(폴더도 파일임)','권한문자')
    - 권한문자 : 10진수 권한
    - ex. 777 : rwxrwxrwx -> 모든 user가 읽기/쓰기/실행 권한을 갖는다
    - tip. 777 권한은 temp 폴더 등이 갖는 권한이고 공격 대상이 될수 있는 권한(부여시 주의해야 함)

In [12]:
client.set_permission('/corona_data/tmp','777')

## hdfs 파일 삭제
- InsecureClient클래스의 delete('삭제할파일(경로 포함)')
    - 삭제가 정상적으로 진행되면 True 반환
    - False 반환인 경우 대부분 파일명 문제임

In [13]:
client.delete('/corona_data/tmp/sido_area.xlsx')

True

############################################################


## 실습 DataPipeline
- DataWarehouse Extract
    - 현재 실행중인 ubuntu server의 HDD -> Hadoop FS로 변경된 공간
        - hadoop설치 : hdfs 구성
    - Extract 과정 거쳐서 hdfs에 data 저장
        1. corona 발생현황(일간데이터) : api 수집(json형식)
        2. corona vaccine 접종현황(일간데이터) : 파일데이터에서 특정일에 대한 데이터 json형식으로 변환 수집
        3. 인구 사회 데이터 : 파일데이터(변환없이 사용)
- Transformation
    -  HDFS에서 데이터 load와서 spark 모듈이용 transformation 진행
        -  DaraWarehouse의 데이터보다 정형화된 데이터로 trans
- DataMart Load
    - transformation 진행된 data를 MySql db에 저장(적재)
- Transformation
    -  DM에서 데이터 load해와서 spark 모듈이용 transformation 진행
        -  DM의 데이터보다 정형화되고 논리적 의미를 포함하는 데이터로 trans
- 운영DB Load
    - transformation 진행된 data를 MySql db에 저장(적재)
    - 운영DB에 적재되는 데이터는 서비스와 연결됨
    - 정형화되고 논리적의미를 포함해야 함

### REST_API로 데이터를 호출해 HDFS에 저장

#### API 데이터 수집 과정
1. 데이터신청
2. site에서 데이터 수집을 위한 url을 알아와야 함
3. site에서 가이드하고 있는 방식(get/post)으로 데이터 요청 진행
    - header와 params를 요구하게됨
        -  생성해서 요청시마다 사용
        -  params에 service key를 포함시켜서 구성
5. 요청된 데이터가 프로그램 변수에 저장되면 이 data를 정리해서 저장\
6. executeRestApi() 요청하고 데이터받아 변수저장 함수를 호출한 곳에 데이터 반환(json형태의 text객체를 반환)

In [14]:
# 데이터요청에 필요한 함수 인수를 받아서
# 데이터 요청하고 전송된 데이터를 반환하는 함수
# 데이터 요청이 실패했을 경우 예외 발생시킴 
# log를 구성해서 예외발생 시 log에 기록되도록 진행
def executeRestApi(method, url, headers, params):  
    # params, headers는 dict로 구성되어야 함
    # raise Exception('응답코드 : 500')  # 예외 테스트
    # err_num = 10/0 # 예외 테스트
    if method == "get": # header가 필요없음 - 파라미터로 사용해야 된다면 보통 {} 빈디렉터리가 전달됨
        res = requests.get(url, params=params, headers=headers)
    else: # post 요청 : header 필요, header는 {}로 구성되어야 함
        res = requests.post(url, data=params, headers=headers)

    if res == None or res.status_code != 200:
        raise Exception('응답코드 : ' + str(res.status_code))
       
    return res.text



### 기준일자 생성 함수
- 요청 data의 기준이 되는 yyyy-mm-dd 형식의 날짜형태 문자열을 생성하는 함수
- 현시점으로부터 며칠전 data인지에 대한 기준으로 날짜 생성
- ex. 어제 data : 오늘부터 1일전
- ex. 1년전 data : 오늘부터 365일전
- ex. 오늘 : 오늘부터 0일전
- ex. 내일 : 오늘부터 -1일전 (이후의 개념)

In [20]:
dt.datetime.now() - dt.timedelta(365)
d = dt.datetime.now()
type(d.month)

datetime.datetime(2024, 1, 18, 4, 40, 35, 918965)

int

In [21]:
# 오늘부터 얼마나 이전 날짜인지
def cal_std_day(befor_day):
    x = dt.datetime.now() - dt.timedelta(befor_day)
    year = x.year
    month = x.month if x.month >= 10 else '0'+ str(x.month)
    day = x.day if x.day >= 10 else '0' + str(x.day)
    return str(year) + '-' + str(month) + '-' + str(day)    


In [26]:
cal_std_day(365*3)
cal_std_day(0)
cal_std_day(-1)

'2022-01-18'

'2025-01-17'

'2025-01-18'

## log 기록

#### logger 
- log 기록하는 프로그램
- log  : 프로그램을 실행하면서 나타나는 일련의 사건(error, event등 물리적으로 발생하는 내용이나 네트워크 송수신중 발생하는 많은 내용)
- 파이썬 패키지 : logging

- log는 외부연결시 오류 관련 확인 위해서 사용자 정의를 기록 해야함
- respAPI는 외부 서버 : log 기록
- 사용자 정의 log 생성
    1. 로그 저장할 dir 필요
    2. 로그 관련 패키지 import
    3. 모듈 활용 객체 logger 생성 (logging.getLogger('로거이름'))
    4. 로그파일 핸들러 생성 (logging.FileHandler('로그파일명.log'))
    5. 로그파일 핸들러 로거에 추가(로거와 로그파일 연결) : logger.addHandler(로그파일핸들러 객체명)

In [27]:
import logging # 로그관련 패키지

In [29]:
co_logger = logging.getLogger('corona_api')
f_handler = logging.FileHandler('./log/rest_api/'+cal_std_day(0)+'.log')
co_logger.addHandler(f_handler)

In [30]:
# 이렇게 문자열로 찍을 수도 있음
# json양식으로 log를 저장하면 다음에 읽어와서 처리하기도 편하다.
# 강제 error 발생시켜서 log에 기록 text
co_logger.error('corona_patient_'+cal_std_day(0)+'.json 다운로드 실패')
# 발생된 에러의 log는 /root/note/log/rest_api/ 디렉터리내 log 파일에서 확인해야 함

### api 호출

In [46]:
# base var 생성
url = 'http://apis.data.go.kr/1352000/ODMS_COVID_04/callCovid04Api'
service_key = 'azOlzHmE12kpPv/6L9CM1ylcu+fqgba5RR/xip8K1OQuncWE3OGzAW5hiE2oKb6fbaMFwkSf8YGy1sytUI5feA=='
# 요청해 응답받은 rest_api data를 hdfs에 저장할 dir
file_dir = '/corona_data/patient/'


- 기본함수
    - executerRestApi(method, url, headers, params)
    - params는 여러 data를 설정해야 함
        - 파라미터 생성 함수 : create_param(std_day)
            - std_day : 날짜가 아닌 오늘부터 얼마나 이전 데이터를 요청할것인에 대한 수치

In [31]:
def create_param(std_day) :
    return {'serviceKey':service_key
        ,'pageNo':1
        ,'numOfRows':'500'
        ,'apiType':'JSON'
        ,'std_day':cal_std_day(std_day)}

In [35]:
create_param(365*3)

{'serviceKey': 'azOlzHmE12kpPv%2F6L9CM1ylcu%2Bfqgba5RR%2Fxip8K1OQuncWE3OGzAW5hiE2oKb6fbaMFwkSf8YGy1sytUI5feA%3D%3D',
 'pageNo': 1,
 'numOfRows': '500',
 'apiType': 'JSON',
 'std_day': '2022-01-18'}

In [57]:
params = create_param(365*3)
#log기록 할 내용 결정(개발자가 결정)
log_dict={
    "is_success":"Fail"
    ,"type":"corona_patient"
    ,"std_day":params['std_day']
    ,"params":params
}

- 날짜별 발생현황 파일 요청
    - 해당파일에 날짜를 파일명에 포함시켜서
    - 정리없이 hdfs 저장

In [52]:
# 요청해서 json 응답받고 hdfs에 저장 - 날짜별로 파일명 구분
try : 
    res=executeRestApi('get',url,{},params) # api에 data 요청후 결과 응답
    file_name='corona_patient_'+cal_std_day(365*3)+'.json' # hdfs에 저장할 파일명 생성
    client.write(file_dir+file_name,res,encoding='utf-8') # hdfs에 저장
except Exception as e: # 일반 python 예외
    log_dict['err_msg']= e.__str__()
    log_json = json.dumps(log_dict, ensure_ascii=False) #log json 형태로 기록
    co_logger.error(log_json) #logger가 발생시키는 에러 - log 기록을 위함  
# print(res)    

In [64]:
print(res)

{"pageNo":"1","resultCode":"00","totalCount":19,"items":[{"gubunEn":"Total","deathCnt":"6210","defCnt":"679030","isolClearCnt":"575763","stdDay":"2022-01-13","localOccCnt":"3776","qurRate":"1315","overFlowCnt":"391","gubunCn":"合计","incDec":"4167","isolIngCnt":"0","gubun":"합계"},{"gubunEn":"Gwangju","deathCnt":"54","defCnt":"9436","isolClearCnt":"8221","stdDay":"2022-01-13","localOccCnt":"182","qurRate":"655","overFlowCnt":"6","gubunCn":"光州","incDec":"188","isolIngCnt":"0","gubun":"광주"},{"gubunEn":"Daegu","deathCnt":"382","defCnt":"24436","isolClearCnt":"22883","stdDay":"2022-01-13","localOccCnt":"119","qurRate":"1024","overFlowCnt":"6","gubunCn":"大邱","incDec":"125","isolIngCnt":"0","gubun":"대구"},{"gubunEn":"Gangwon-do","deathCnt":"110","defCnt":"13368","isolClearCnt":"12342","stdDay":"2022-01-13","localOccCnt":"82","qurRate":"869","overFlowCnt":"7","gubunCn":"江原","incDec":"89","isolIngCnt":"0","gubun":"강원"},{"gubunEn":"Jeollanam-do","deathCnt":"44","defCnt":"7174","isolClearCnt":"5884",

In [54]:
cal_std_day(365*3)
cal_std_day(367*3)

'2022-01-18'

'2022-01-12'

In [59]:
# ETL 연습용 data hdfs에 저장
for i in range(365*3, 367*3) :
    params = create_param(i)
    print(params['std_day'])
    try : 
        res=executeRestApi('get',url,{},params)
        file_name='corona_patient_'+cal_std_day(i)+'.json'
        client.write(file_dir+file_name,res,encoding='utf-8') 
    except Exception as e:
        log_dict['err_msg']= e.__str__()
        log_json = json.dumps(log_dict, ensure_ascii=False) 
        co_logger.error(log_json) 

2022-01-18
2022-01-17
2022-01-16
2022-01-15
2022-01-14
2022-01-13


- 2022-01-13 ~ 2022-01-18 코로나 발생현황 data hdfs에 저장
- Json 형식으로 저장

## corona_vaccine 접종현황 data
- 파일로 제공
- 파일 data 읽어서 Json 형식으로 변경한 후에 hdfs에 저장
- 필요 data 추출 후 정리 형식 변환

In [60]:
import pandas as pd

In [75]:
# 광역 시도별 인구 data
pop = pd.read_csv('data/corona_data/sido_population.csv',encoding='CP949')
pop.index
pop.head()

RangeIndex(start=0, stop=18, step=1)

,loc,total,male,female
0,전국,52739009,26360035,26378974
1,서울,9736027,4721977,5014050
2,부산,3396109,1663619,1732490
3,대구,2412642,1189901,1222741
4,인천,3014739,1513277,1501462


In [62]:
vac_v1 = pd.read_csv('data/corona_data/Covid19Vaccine.csv', index_col=0)
vac_v1.head(5)

,sido,firstCnt,secondCnt,totalFirstCnt,totalSecondCnt,accumulatedFirstCnt,accumulatedSecondCnt,id,thirdCnt,totalThirdCnt,accumulatedThirdCnt
baseDate,,,,,,,,,,,
2022-11-09 00:00:00,제주특별자치도,6.0,5.0,591064,585442.0,591058,585437.0,10176,38.0,439144.0,439106.0
2022-11-09 00:00:00,전국,175.0,224.0,45123225,44693321.0,45123050,44693097.0,10159,1678.0,33673146.0,33671468.0
2022-11-09 00:00:00,서울특별시,28.0,43.0,8335326,8257811.0,8335298,8257768.0,10160,482.0,6076911.0,6076429.0
2022-11-09 00:00:00,부산광역시,8.0,14.0,2879908,2851376.0,2879900,2851362.0,10161,136.0,2140909.0,2140773.0
2022-11-09 00:00:00,대구광역시,8.0,8.0,2018612,1995791.0,2018604,1995783.0,10162,56.0,1405219.0,1405163.0


In [63]:
vac_v1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10158 entries, 2022-11-09 00:00:00 to 2021-03-11 00:00:00
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   sido                  10158 non-null  object 
 1   firstCnt              9839 non-null   float64
 2   secondCnt             9339 non-null   float64
 3   totalFirstCnt         10158 non-null  int64  
 4   totalSecondCnt        9867 non-null   float64
 5   accumulatedFirstCnt   10158 non-null  int64  
 6   accumulatedSecondCnt  9851 non-null   float64
 7   id                    10158 non-null  int64  
 8   thirdCnt              6191 non-null   float64
 9   totalThirdCnt         6261 non-null   float64
 10  accumulatedThirdCnt   6243 non-null   float64
dtypes: float64(7), int64(3), object(1)
memory usage: 952.3+ KB


- 백신 데이터 광역시도명이 기 수집한 데이터의 광역시도명과 다르기때문에
    - 백신 데이터를 변경 후 파일 저장

In [66]:
# 원본 data 추출 : 중복 없이 추출
vac_v1[['sido']].drop_duplicates()['sido']

baseDate
2022-11-09 00:00:00    제주특별자치도
2022-11-09 00:00:00         전국
2022-11-09 00:00:00      서울특별시
2022-11-09 00:00:00      부산광역시
2022-11-09 00:00:00      대구광역시
2022-11-09 00:00:00      광주광역시
2022-11-09 00:00:00      인천광역시
2022-11-09 00:00:00      대전광역시
2022-11-09 00:00:00        경기도
2022-11-09 00:00:00    세종특별자치시
2022-11-09 00:00:00       경상남도
2022-11-09 00:00:00       경상북도
2022-11-09 00:00:00       전라남도
2022-11-09 00:00:00       전라북도
2022-11-09 00:00:00       충청남도
2022-11-09 00:00:00       충청북도
2022-11-09 00:00:00        강원도
2022-11-09 00:00:00      울산광역시
2021-04-26 00:00:00         기타
Name: sido, dtype: object

In [69]:
before = [ sido for sido in vac_v1[['sido']].drop_duplicates()['sido']]
before.sort()
before.remove('기타')
print(before)

['강원도', '경기도', '경상남도', '경상북도', '광주광역시', '대구광역시', '대전광역시', '부산광역시', '서울특별시', '세종특별자치시', '울산광역시', '인천광역시', '전국', '전라남도', '전라북도', '제주특별자치도', '충청남도', '충청북도']


In [77]:
pop['loc'] # 시리즈

0     전국
1     서울
2     부산
3     대구
4     인천
5     광주
6     대전
7     울산
8     세종
9     경기
10    강원
11    충북
12    충남
13    전북
14    전남
15    경북
16    경남
17    제주
Name: loc, dtype: object

In [79]:
after = [sido for sido in pop['loc']]
after.sort()
print(after)

['강원', '경기', '경남', '경북', '광주', '대구', '대전', '부산', '서울', '세종', '울산', '인천', '전국', '전남', '전북', '제주', '충남', '충북']


In [80]:
for i in range(0, len(after)) :
    vac_v1['sido'] = vac_v1['sido'].replace(before[i], after[i])

In [81]:
vac_v1.head(50)

,sido,firstCnt,secondCnt,totalFirstCnt,totalSecondCnt,accumulatedFirstCnt,accumulatedSecondCnt,id,thirdCnt,totalThirdCnt,accumulatedThirdCnt
baseDate,,,,,,,,,,,
2022-11-09 00:00:00,제주,6.0,5.0,591064,585442.0,591058,585437.0,10176,38.0,439144.0,439106.0
2022-11-09 00:00:00,전국,175.0,224.0,45123225,44693321.0,45123050,44693097.0,10159,1678.0,33673146.0,33671468.0
2022-11-09 00:00:00,서울,28.0,43.0,8335326,8257811.0,8335298,8257768.0,10160,482.0,6076911.0,6076429.0
2022-11-09 00:00:00,부산,8.0,14.0,2879908,2851376.0,2879900,2851362.0,10161,136.0,2140909.0,2140773.0
2022-11-09 00:00:00,대구,8.0,8.0,2018612,1995791.0,2018604,1995783.0,10162,56.0,1405219.0,1405163.0
2022-11-09 00:00:00,광주,6.0,6.0,1261987,1250574.0,1261981,1250568.0,10163,29.0,968976.0,968947.0
2022-11-09 00:00:00,인천,8.0,8.0,2574241,2550089.0,2574233,2550081.0,10164,99.0,1915365.0,1915266.0
2022-11-09 00:00:00,대전,6.0,1.0,1245479,1233377.0,1245473,1233376.0,10165,40.0,909142.0,909102.0
2022-11-09 00:00:00,경기,54.0,54.0,11837228,11723312.0,11837174,11723258.0,10167,476.0,8708303.0,8707827.0


- 2022-01-18 에 해당하는 data를 vac_v1에서 추출

In [97]:
cal_std_day(365*3)

'2022-01-18'

In [90]:
# T/F를 이용한 논리 indexing
# loc에 논리 인덱싱 행 index만 구성하면 레코드를 추출
finVac = vac_v1.loc[vac_v1.index.str.contains(cal_std_day(365*3)]
finVac.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18 entries, 2022-01-18 00:00:00 to 2022-01-18 00:00:00
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   sido                  18 non-null     object 
 1   firstCnt              18 non-null     float64
 2   secondCnt             18 non-null     float64
 3   totalFirstCnt         18 non-null     int64  
 4   totalSecondCnt        18 non-null     float64
 5   accumulatedFirstCnt   18 non-null     int64  
 6   accumulatedSecondCnt  18 non-null     float64
 7   id                    18 non-null     int64  
 8   thirdCnt              18 non-null     float64
 9   totalThirdCnt         18 non-null     float64
 10  accumulatedThirdCnt   18 non-null     float64
dtypes: float64(7), int64(3), object(1)
memory usage: 1.7+ KB


In [92]:
finVac.head()

,sido,firstCnt,secondCnt,totalFirstCnt,totalSecondCnt,accumulatedFirstCnt,accumulatedSecondCnt,id,thirdCnt,totalThirdCnt,accumulatedThirdCnt
baseDate,,,,,,,,,,,
2022-01-18 00:00:00,광주,523.0,2469.0,1247764,1220758.0,1247241,1218289.0,5648,11353.0,682394.0,671041.0
2022-01-18 00:00:00,울산,382.0,2280.0,956895,933311.0,956513,931031.0,5647,9470.0,476692.0,467222.0
2022-01-18 00:00:00,인천,985.0,3663.0,2532911,2484196.0,2531926,2480533.0,5646,23198.0,1330273.0,1307075.0
2022-01-18 00:00:00,대전,422.0,2197.0,1231435,1203724.0,1231013,1201527.0,5645,10221.0,628469.0,618248.0
2022-01-18 00:00:00,대구,893.0,4524.0,1997407,1945813.0,1996514,1941289.0,5644,15716.0,978794.0,963078.0


In [93]:
# 레코드 추출 - iloc[위치값]
finVac.iloc[4]

sido                           대구
firstCnt                    893.0
secondCnt                  4524.0
totalFirstCnt             1997407
totalSecondCnt          1945813.0
accumulatedFirstCnt       1996514
accumulatedSecondCnt    1941289.0
id                           5644
thirdCnt                  15716.0
totalThirdCnt            978794.0
accumulatedThirdCnt      963078.0
Name: 2022-01-18 00:00:00, dtype: object

#### Json 구성

In [91]:
# hdfs에 저장할 data
cols = ['loc', 'v1', 'v2', 'v3']
data = []

In [95]:
for idx in range(0, len(finVac.index)) :
    tuple_t=[]
    tmp = finVac.iloc[idx]
    tuple_t.append(str(tmp.sido))
    tuple_t.append(str(tmp.totalFirstCnt))
    tuple_t.append(str(tmp.totalSecondCnt))
    tuple_t.append(str(tmp.thirdCnt))
    # print(tuple_t)
    data.append(dict(zip(cols,tuple_t)))

In [96]:
# 실제 data
data

[{'loc': '광주', 'v1': '1247764', 'v2': '1220758.0', 'v3': '11353.0'},
 {'loc': '울산', 'v1': '956895', 'v2': '933311.0', 'v3': '9470.0'},
 {'loc': '인천', 'v1': '2532911', 'v2': '2484196.0', 'v3': '23198.0'},
 {'loc': '대전', 'v1': '1231435', 'v2': '1203724.0', 'v3': '10221.0'},
 {'loc': '대구', 'v1': '1997407', 'v2': '1945813.0', 'v3': '15716.0'},
 {'loc': '부산', 'v1': '2849093', 'v2': '2788089.0', 'v3': '22579.0'},
 {'loc': '서울', 'v1': '8239400', 'v2': '8090244.0', 'v3': '70311.0'},
 {'loc': '전국', 'v1': '44505276', 'v2': '43582128.0', 'v3': '384948.0'},
 {'loc': '세종', 'v1': '290455', 'v2': '282507.0', 'v3': '2330.0'},
 {'loc': '경기', 'v1': '11652853', 'v2': '11414202.0', 'v3': '104972.0'},
 {'loc': '제주', 'v1': '580579', 'v2': '566811.0', 'v3': '4782.0'},
 {'loc': '경남', 'v1': '2848083', 'v2': '2783896.0', 'v3': '22358.0'},
 {'loc': '경북', 'v1': '2260963', 'v2': '2202814.0', 'v3': '18629.0'},
 {'loc': '전남', 'v1': '1632045', 'v2': '1602479.0', 'v3': '14141.0'},
 {'loc': '전북', 'v1': '1574779', 'v2':

In [98]:
# data를 설명하는 메타 데이터 포함한 최종 결과 생성
res = {
    'meta' : {
        'desc':'지역별 코로나 예방접종 인구 현황',
        'cols':{
            'loc':'지역'
            ,'v1':'1차 접종 누적'
            ,'v2':'2차 접종 누적'
            ,'v3':'3차 접종 누적'
        },
        'stdDay':cal_std_day(365*3)
    },
    'data':data
}

In [99]:
res

{'meta': {'desc': '지역별 코로나 예방접종 인구 현황',
  'cols': {'loc': '지역', 'v1': '1차 접종 누적', 'v2': '2차 접종 누적', 'v3': '3차 접종 누적'},
  'stdDay': '2022-01-18'},
 'data': [{'loc': '광주', 'v1': '1247764', 'v2': '1220758.0', 'v3': '11353.0'},
  {'loc': '울산', 'v1': '956895', 'v2': '933311.0', 'v3': '9470.0'},
  {'loc': '인천', 'v1': '2532911', 'v2': '2484196.0', 'v3': '23198.0'},
  {'loc': '대전', 'v1': '1231435', 'v2': '1203724.0', 'v3': '10221.0'},
  {'loc': '대구', 'v1': '1997407', 'v2': '1945813.0', 'v3': '15716.0'},
  {'loc': '부산', 'v1': '2849093', 'v2': '2788089.0', 'v3': '22579.0'},
  {'loc': '서울', 'v1': '8239400', 'v2': '8090244.0', 'v3': '70311.0'},
  {'loc': '전국', 'v1': '44505276', 'v2': '43582128.0', 'v3': '384948.0'},
  {'loc': '세종', 'v1': '290455', 'v2': '282507.0', 'v3': '2330.0'},
  {'loc': '경기', 'v1': '11652853', 'v2': '11414202.0', 'v3': '104972.0'},
  {'loc': '제주', 'v1': '580579', 'v2': '566811.0', 'v3': '4782.0'},
  {'loc': '경남', 'v1': '2848083', 'v2': '2783896.0', 'v3': '22358.0'},
  {'loc':

In [100]:
file_dir_vac = '/corona_data/vaccine/'
file_name = 'corona_vaccine_'+cal_std_day(365*3)+'.json'
client.write(file_dir_vac+file_name,json.dumps(res, ensure_ascii=False),encoding='utf-8')

In [101]:
# spark 모듈(read.json) : df로 지연연산
file_dir_vac = '/corona_data/vaccine/'
file_name = 'corona_vaccine_'+cal_std_day(365*3)+'.json'
vaccine=spark.read.json(file_dir_vac+file_name)
vaccine.show()

[Stage 1:>                                                          (0 + 1) / 1]

+----------------------+---------------------------+
|                  data|                       meta|
+----------------------+---------------------------+
|[{광주, 1247764, 12...|{{지역, 1차 접종 누적, 2...|
+----------------------+---------------------------+



In [ ]:
### 파일 읽어오기
vac_v1 = pd.read_csv('data/corona_data/Covid19VaccineV2.csv',index_col=0)


In [ ]:
# 전체 코드
cols = ['loc', 'v1', 'v2', 'v3']
data = []
finVac = vac_v1.loc[vac_v1.index.str.contains(cal_std_day(365*3)]

for idx in range(0, len(finVac.index)) :
    tuple_t=[]
    tmp = finVac.iloc[idx]
    tuple_t.append(str(tmp.sido))
    tuple_t.append(str(tmp.totalFirstCnt))
    tuple_t.append(str(tmp.totalSecondCnt))
    tuple_t.append(str(tmp.thirdCnt))
    # print(tuple_t)
    data.append(dict(zip(cols,tuple_t)))

res = {
    'meta' : {
        'desc':'지역별 코로나 예방접종 인구 현황',
        'cols':{
            'loc':'지역'
            ,'v1':'1차 접종 누적'
            ,'v2':'2차 접종 누적'
            ,'v3':'3차 접종 누적'
        },
        'stdDay':cal_std_day(365*3)
    },
    'data':data
}    

file_dir_vac = '/corona_data/vaccine/'
file_name = 'corona_vaccine_'+cal_std_day(365*3)+'.json'
client.write(file_dir_vac+file_name,json.dumps(res, ensure_ascii=False),encoding='utf-8')

In [104]:
# 연습용 파일 생성 -> hdfs에 저장
for i in range(365*3, 367*3) :
    print(cal_std_day(i))
    cols = ['loc', 'v1', 'v2', 'v3']
    data = []
    finVac = vac_v1.loc[vac_v1.index.str.contains(cal_std_day(i))]
    
    for idx in range(0, len(finVac.index)) :
        tuple_t=[]
        tmp = finVac.iloc[idx]
        tuple_t.append(str(tmp.sido))
        tuple_t.append(str(tmp.totalFirstCnt))
        tuple_t.append(str(tmp.totalSecondCnt))
        tuple_t.append(str(tmp.thirdCnt))
        # print(tuple_t)
        data.append(dict(zip(cols,tuple_t)))
    
    res = {
        'meta' : {
            'desc':'지역별 코로나 예방접종 인구 현황',
            'cols':{
                'loc':'지역'
                ,'v1':'1차 접종 누적'
                ,'v2':'2차 접종 누적'
                ,'v3':'3차 접종 누적'
            },
            'stdDay':cal_std_day(365*3)
        },
        'data':data
    }    
    try : 
        file_dir_vac = '/corona_data/vaccine/'
        file_name = 'corona_vaccine_'+cal_std_day(i)+'.json'
        client.write(file_dir_vac+file_name,json.dumps(res, ensure_ascii=False),encoding='utf-8')
    except Exception as e :
        log_dict = {
            'type':'/corona_data/vaccine/'
            ,'err_msg':e.__str__()}
        log_json = json.dumps(log_dict, ensure_ascii=False)
        co_logger.error(log_json)

ERROR:corona_api:{"type": "/corona_data/vaccine/", "err_msg": "/corona_data/vaccine/corona_vaccine_2022-01-18.json for client 127.0.0.1 already exists\n\tat org.apache.hadoop.hdfs.server.namenode.FSDirWriteFileOp.startFile(FSDirWriteFileOp.java:388)\n\tat org.apache.hadoop.hdfs.server.namenode.FSNamesystem.startFileInt(FSNamesystem.java:2819)\n\tat org.apache.hadoop.hdfs.server.namenode.FSNamesystem.startFile(FSNamesystem.java:2713)\n\tat org.apache.hadoop.hdfs.server.namenode.NameNodeRpcServer.create(NameNodeRpcServer.java:830)\n\tat org.apache.hadoop.hdfs.protocolPB.ClientNamenodeProtocolServerSideTranslatorPB.create(ClientNamenodeProtocolServerSideTranslatorPB.java:504)\n\tat org.apache.hadoop.hdfs.protocol.proto.ClientNamenodeProtocolProtos$ClientNamenodeProtocol$2.callBlockingMethod(ClientNamenodeProtocolProtos.java)\n\tat org.apache.hadoop.ipc.ProtobufRpcEngine2$Server$ProtoBufRpcInvoker.call(ProtobufRpcEngine2.java:621)\n\tat org.apache.hadoop.ipc.ProtobufRpcEngine2$Server$Proto

2022-01-18
2022-01-17
2022-01-16
2022-01-15
2022-01-14
2022-01-13
